TSG062 - Get tail of all previous container logs for pods in BDC namespace
==========================================================================

Steps
-----

### Parameters

In [ ]:
tail_lines = 10000
coalesce_duplicates = True

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Get logs for previous instance of all containers in Big Data Cluster namespace

In [ ]:
pod_list = api.list_namespaced_pod(namespace)

pod_names = [pod.metadata.name for pod in pod_list.items]

print('Scanning pods: ' + ', '.join(pod_names))

for pod in pod_list.items:
    print("*** %s\t%s\t%s" % (pod.metadata.name,
                        pod.status.phase,
                        pod.status.pod_ip))

    container_names = [container.name for container in pod.spec.containers]

    for container in container_names:
        print (f"POD: {pod.metadata.name} / CONTAINER: {container}")
        try:
            logs = api.read_namespaced_pod_log(pod.metadata.name, namespace, container=container, tail_lines=tail_lines, previous=True)

            if coalesce_duplicates:
                previous_line = ""
                duplicates = 1
                for line in logs.split('\n'):
                    if line[27:] != previous_line[27:]:
                        if duplicates != 1:
                            print(f"\t{previous_line} (x{duplicates})")
                        print(f"\t{line}")
                        duplicates = 1
                    else:
                        duplicates = duplicates + 1

                    previous_line = line
            else:
                print(logs)
        except Exception:
            print (f"Failed to get LOGS for CONTAINER: {container} in POD: {pod.metadata.name}")

In [ ]:
print("Notebook execution is complete.")

Related
-------

- [TSG061 - Get tail of all container logs for pods in BDC namespace](../log-files/tsg061-tail-bdc-container-logs.ipynb)
